## Objective

download all genomes in an NCBI accession list table
e.g. [this table](https://www.ncbi.nlm.nih.gov/labs/virus/vssi/#/virus?SeqType_s=Nucleotide&VirusLineage_ss=SARS-CoV-2,%20taxid:2697049)

In [1]:
# papermill parameters
base="$(homedir())/workspace"
project="sars-cov2-pangenome-analysis"

accession_table_file = "$(base)/$(project)/metadata/sequences.csv"
output_directory = "$(base)/$(project)/data/sequences"

"/home/jovyan/workspace/sars-cov2-pangenome-analysis/data/sequences"

## Materials, Methods, and Functions

In [2]:
mkpath(output_directory)

"/home/jovyan/workspace/sars-cov2-pangenome-analysis/data/sequences"

In [3]:
import Pkg
Pkg.update()

pkgs = [
    "Revise",
    "DataFrames",
    "uCSV",
    "ProgressMeter",
    "FASTX"
]

for pkg in pkgs
    try
        eval(Meta.parse("import $pkg"))
    catch
        Pkg.add(pkg)
        eval(Meta.parse("import $pkg"))
    end
end

# Pkg.add(url="https://github.com/cjprybol/Mycelia.git", rev="master")
import Mycelia

    Updating registry at `/opt/julia/registries/General`
   Installed StatsPlots ────────── v0.15.4
   Installed DocStringExtensions ─ v0.9.2
   Installed BioAlignments ─────── v3.0.0
   Installed FileIO ────────────── v1.16.0
   Installed HTTP ──────────────── v1.5.1
   Installed Plots ─────────────── v1.35.5
   Installed DataFrames ────────── v1.4.2
   Installed Distributions ─────── v0.25.76
   Installed GLM ───────────────── v1.8.1
   Installed MbedTLS ───────────── v1.1.7
   Installed DiffRules ─────────── v1.12.0
   Installed MKL_jll ───────────── v2022.2.0+0
   Installed DataAPI ───────────── v1.12.0
   Installed QuadGK ────────────── v2.6.0
   Installed AbstractTrees ─────── v0.4.3
   Installed RecipesBase ───────── v1.3.1
   Installed CodeTracking ──────── v1.1.1
   Installed ArrayInterfaceCore ── v0.1.23
   Installed PrettyTables ──────── v2.1.2
   Installed StructArrays ──────── v0.6.13
   Installed RecipesPipeline ───── v0.6.7
   Installed SentinelArrays ────── v1.3.16
   I

In [ ]:
# @time accession_table = DataFrames.DataFrame(uCSV.read(accession_table_file, typedetectrows=100, header=1, quotes='"')...)

In [4]:
p = ProgressMeter.Progress(countlines(accession_table_file))

ProgressMeter.Progress(6043674, ReentrantLock(nothing, Base.GenericCondition{Base.Threads.SpinLock}(Base.InvasiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), 0), 0.1, 0, 1.667060176241368e9, 1.667060176241368e9, 1.667060176241368e9, false, "Progress: ", nothing, ProgressMeter.BarGlyphs('|', '█', ['▏', '▎', '▍', '▌', '▋', '▊', '▉'], ' ', '|'), :green, IJulia.IJuliaStdio{Base.PipeEndpoint}(IOContext(Base.PipeEndpoint(RawFD(42) open, 0 bytes waiting))), 0, 0, 0, true, false, 1, 1, Int64[])

In [ ]:
# sort files by reverse size
# this is helpful to track down incomplete sequences
# can also determine the size of each genome
# then determine if there is a good cutoff at which to just remove the incomplete ones
# ls -lShr

# 2_262_407
# figure out how many we have so far
# ls -1 ~/workspace/sars-cov2-pangenome-analysis/sequences/ | wc -l

table_io = open(accession_table_file)
header = readline(table_io)
# ProgressMeter.@showprogress for accession in accession_table[!, "Accession"]
for line in eachline(table_io)
    accession = split(line, ',')[1]
    outfile = joinpath(output_directory, "$(accession).fna")
    if !isfile(outfile) || (filesize(outfile) == 0)
        fasta_records = collect(Mycelia.get_sequence(db="nuccore", accession=accession))
        open(outfile, "w") do io
            fastx_io = FASTX.FASTA.Writer(io)
            for record in fasta_records
                write(fastx_io, record)
            end
            close(fastx_io)
        end
    end
    ProgressMeter.next!(p)
end
table_io

Progress:   0%|▏                                        |  ETA: 5 days, 21:29:04